In [3]:
%pip install ptflops

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#%pip install torch
#%pip install torchvision
# compile pytorch with cuda support
#%pip install torch torchvision torchaudio cudatoolkit=11.1 -f https://download.pytorch.org/whl/cu111/torch_stable.html
%pip install cuda-python
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from ptflops import get_model_complexity_info

In [20]:
# Step 1: Dataset Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=20, shuffle=True, num_workers=0)
#does the tensor have CUDA enabled?

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=20, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
# Step 2: Model Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # TODO ändra lager
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        self.fc2 = nn.Linear(500, 100)


    def forward(self, x):
        F = torch.nn.functional
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = CNN()

if torch.cuda.is_available():
    net.to(device)
    net.cuda()

In [27]:
with torch.cuda.device(0):
  macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True, backend='pytorch')
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

CNN(
  586.18 k, 100.000% Params, 3.42 MMac, 98.338% MACs, 
  (conv1): Conv2d(448, 0.076% Params, 458.75 KMac, 13.185% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(0, 0.000% Params, 28.67 KMac, 0.824% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4.64 k, 0.792% Params, 1.19 MMac, 34.139% MACs, 16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(18.5 k, 3.155% Params, 1.18 MMac, 34.021% MACs, 32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(512.5 k, 87.430% Params, 512.5 KMac, 14.729% MACs, in_features=1024, out_features=500, bias=True)
  (fc2): Linear(50.1 k, 8.547% Params, 50.1 KMac, 1.440% MACs, in_features=500, out_features=100, bias=True)
)
Computational complexity:       3.48 MMac
Number of parameters:           586.18 k


In [28]:
if torch.cuda.is_available():
  print ("We be using CUDA")
else:
  print ("We be using CPU")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

We be using CUDA


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
)

In [36]:
# Step 3: Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

net.train()

for epoch in range(20):  # Loop over the dataset multiple times
# TODO ändra epoker
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.671
[2,  2000] loss: 0.574
[3,  2000] loss: 0.536
[4,  2000] loss: 0.495
[5,  2000] loss: 0.456
[6,  2000] loss: 0.427
[7,  2000] loss: 0.408
[8,  2000] loss: 0.381
[9,  2000] loss: 0.366
[10,  2000] loss: 0.370
[11,  2000] loss: 0.350
[12,  2000] loss: 0.343
[13,  2000] loss: 0.324
[14,  2000] loss: 0.322
[15,  2000] loss: 0.316
[16,  2000] loss: 0.310
[17,  2000] loss: 0.306
[18,  2000] loss: 0.302
[19,  2000] loss: 0.291
[20,  2000] loss: 0.287
Finished Training


In [37]:
# Step 4: Testing
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 33 %
